In [2]:
!pip install -q chromadb langchain sentence-transformers transformers

# Important Library

In [3]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.3/496.3 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.0/342.0 kB 25.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.11
    Uninstalling langchain-text-splitters-0.3.11:
      Successfully uninstalled langchain-text-splitters-0.3.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1

In [4]:
import pandas as pd
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.chains import RetrievalQA
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain_community.llms import HuggingFacePipeline
import os, shutil

2026-02-10 07:34:59.549244: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770708899.734347      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770708899.786922      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770708900.248022      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770708900.248061      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770708900.248064      55 computation_placer.cc:177] computation placer alr

ModuleNotFoundError: No module named 'langchain_core.memory'

In [1]:
import pandas as pd


# Load & Preprocess Review Data

In [7]:
# Example Kaggle CSV path
df = pd.read_csv("/kaggle/input/restaurant-review/dhaka_restaurants.csv")
df.head()

,business_name,business_address,business_phone,business_website,business_rating,business_total_reviews,reviewer_name,review_rating,review_date,review_text,review_additional_info
0,Izumi Japanese Kitchen,"House 24 C, Rd 119, Dhaka 1212, Bangladesh",+880 1933-446677,https://m.facebook.com/izumiBD/,4.5,2233,"{'name': 'Raunak Maskay', 'thumbnail': 'https:...",5.0,a month ago,"Izumi Japanese Kitchen in Gulshan, Dhaka is on...",NaN
1,Izumi Japanese Kitchen,"House 24 C, Rd 119, Dhaka 1212, Bangladesh",+880 1933-446677,https://m.facebook.com/izumiBD/,4.5,2233,"{'name': 'Dewan Asif', 'thumbnail': 'https://l...",5.0,4 months ago,Izumi Japanese Kitchen is a great place for re...,NaN
2,Izumi Japanese Kitchen,"House 24 C, Rd 119, Dhaka 1212, Bangladesh",+880 1933-446677,https://m.facebook.com/izumiBD/,4.5,2233,"{'name': 'Dr. Mehruba Mona', 'thumbnail': 'htt...",5.0,Edited 8 months ago,One of the authentic Japanese restaurant in Dh...,NaN
3,Izumi Japanese Kitchen,"House 24 C, Rd 119, Dhaka 1212, Bangladesh",+880 1933-446677,https://m.facebook.com/izumiBD/,4.5,2233,"{'name': 'Md Kamrul Islam', 'thumbnail': 'http...",5.0,5 months ago,Izumi is one of the best authentic Japanese re...,NaN
4,Izumi Japanese Kitchen,"House 24 C, Rd 119, Dhaka 1212, Bangladesh",+880 1933-446677,https://m.facebook.com/izumiBD/,4.5,2233,"{'name': 'Danielle W', 'thumbnail': 'https://l...",5.0,8 months ago,"What a chic restaurant. Hidden in Gulshan, thi...",NaN


In [9]:
df_cleaned = df.dropna(subset=["review_text"])
df_cleaned = df_cleaned[df_cleaned["review_text"].apply(lambda x: isinstance(x, str))]

df["business_address"] = df["business_address"].astype(str).str.replace(
    r"^\s*|\s*\n?\s*$", "", regex=True
)

from langchain.docstore.document import Document

docs = [
    Document(
        page_content=row["review_text"],
        metadata={
            "address": row.get("business_address", "Unknown"),
            "rating": float(row.get("review_rating", 0))
        }
    )
    for _, row in df_cleaned.iterrows()
]

print(f"✅ {len(docs)} documents prepared.")


✅ 988 documents prepared.


# # # #    **Basic Imports (SAFE)**

In [10]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import torch
from transformers import pipeline


2026-02-10 16:24:14.728212: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770740655.112951      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770740655.229452      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770740656.125278      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770740656.125321      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770740656.125324      55 computation_placer.cc:177] computation placer alr

# # # Load & Basic Clean (EXTEND YOUR EXISTING CELL)****  

In [11]:
# ⚠️ Do NOT reload CSV again – df already loaded correctly above
# df already exists

# 1️⃣ Drop rows where review_text is missing
df_cleaned = df.dropna(subset=["review_text"])

# 2️⃣ Keep only string-type reviews
df_cleaned = df_cleaned[
    df_cleaned["review_text"].apply(lambda x: isinstance(x, str))
]

# 3️⃣ Basic text cleaning
df_cleaned["review_text"] = (
    df_cleaned["review_text"]
    .str.strip()
    .str.replace(r"\s+", " ", regex=True)
)

# 4️⃣ (Optional but safe) Clean address column
df_cleaned["business_address"] = (
    df_cleaned["business_address"]
    .astype(str)
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

# 5️⃣ Sanity check
df_cleaned.head()


,business_name,business_address,business_phone,business_website,business_rating,business_total_reviews,reviewer_name,review_rating,review_date,review_text,review_additional_info
0,Izumi Japanese Kitchen,"House 24 C, Rd 119, Dhaka 1212, Bangladesh",+880 1933-446677,https://m.facebook.com/izumiBD/,4.5,2233,"{'name': 'Raunak Maskay', 'thumbnail': 'https:...",5.0,a month ago,"Izumi Japanese Kitchen in Gulshan, Dhaka is on...",NaN
1,Izumi Japanese Kitchen,"House 24 C, Rd 119, Dhaka 1212, Bangladesh",+880 1933-446677,https://m.facebook.com/izumiBD/,4.5,2233,"{'name': 'Dewan Asif', 'thumbnail': 'https://l...",5.0,4 months ago,Izumi Japanese Kitchen is a great place for re...,NaN
2,Izumi Japanese Kitchen,"House 24 C, Rd 119, Dhaka 1212, Bangladesh",+880 1933-446677,https://m.facebook.com/izumiBD/,4.5,2233,"{'name': 'Dr. Mehruba Mona', 'thumbnail': 'htt...",5.0,Edited 8 months ago,One of the authentic Japanese restaurant in Dh...,NaN
3,Izumi Japanese Kitchen,"House 24 C, Rd 119, Dhaka 1212, Bangladesh",+880 1933-446677,https://m.facebook.com/izumiBD/,4.5,2233,"{'name': 'Md Kamrul Islam', 'thumbnail': 'http...",5.0,5 months ago,Izumi is one of the best authentic Japanese re...,NaN
4,Izumi Japanese Kitchen,"House 24 C, Rd 119, Dhaka 1212, Bangladesh",+880 1933-446677,https://m.facebook.com/izumiBD/,4.5,2233,"{'name': 'Danielle W', 'thumbnail': 'https://l...",5.0,8 months ago,"What a chic restaurant. Hidden in Gulshan, thi...",NaN


# # # Multilingual Sentiment Model****  

In [12]:
from transformers import pipeline
import torch

sentiment_model = pipeline(
    task="sentiment-analysis",
    model="cardiffnlp/twitter-xlm-roberta-base-sentiment",
    device=0 if torch.cuda.is_available() else -1
)



config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Device set to use cuda:0


# # # Overall Sentiment Inference****  

In [13]:
from tqdm import tqdm
tqdm.pandas()

def get_overall_sentiment(text):
    try:
        result = sentiment_model(text[:512])[0]
        return result["label"], result["score"]
    except Exception:
        return "UNKNOWN", 0.0

df_cleaned[["overall_sentiment", "sentiment_confidence"]] = (
    df_cleaned["review_text"]
    .progress_apply(lambda x: pd.Series(get_overall_sentiment(x)))
)



100%|██████████| 988/988 [00:10<00:00, 96.16it/s] 


# # #  Aspect-Based Sentiment (ADVANCED CORE)

In [15]:
# ================================
# Aspect-Based Sentiment (FIXED + ADVANCED)
# ================================

ASPECTS = {
    "food": ["food", "taste", "meal", "coffee", "burger", "pizza", "খাবার"],
    "service": ["service", "staff", "waiter", "manager", "সার্ভিস"],
    "price": ["price", "cost", "expensive", "cheap", "দাম"],
    "ambience": ["ambience", "environment", "atmosphere", "place", "পরিবেশ"]
}

def aspect_sentiment(text, overall_label):
    """
    Use ONE sentiment inference per review
    Reuse it across all aspects
    """
    text_lower = text.lower()
    result = {}

    for aspect, keywords in ASPECTS.items():
        if any(k in text_lower for k in keywords):
            result[aspect] = overall_label
        else:
            result[aspect] = None   # cleaner than NOT_MENTIONED

    return result


# Apply on CLEANED dataframe
aspect_df = df_cleaned.progress_apply(
    lambda row: pd.Series(
        aspect_sentiment(
            row["review_text"],
            row["overall_sentiment"]
        )
    ),
    axis=1
)

# Merge back
df_cleaned = pd.concat([df_cleaned, aspect_df], axis=1)

df_cleaned[
    ["review_text", "overall_sentiment", "food", "service", "price", "ambience"]
].head()

df_cleaned[['overall_sentiment']].value_counts()



100%|██████████| 988/988 [00:00<00:00, 5862.77it/s]


overall_sentiment
positive             736
negative             213
neutral               39
Name: count, dtype: int64

# # # Rating ↔ Text Conflict Detection****  

In [21]:
def detect_conflict(row):
    rating = row.get("review_rating", 0)
    sentiment = row.get("overall_sentiment")

    if rating >= 4 and sentiment == "negative":
        return "Hidden Dissatisfaction"

    if rating <= 2 and sentiment == "positive":
        return "Politeness Bias"

    if rating == 3 and sentiment in ["positive", "negative"]:
        return "Ambiguous Experience"

    return "No Conflict"

df_cleaned["rating_sentiment_conflict"] = df_cleaned.apply(
    detect_conflict, axis=1
)
df_cleaned[
    ["review_text", "review_rating", "overall_sentiment", "rating_sentiment_conflict"]
].head(10)
df_cleaned["rating_sentiment_conflict"].value_counts()



rating_sentiment_conflict
No Conflict               810
Ambiguous Experience       87
Hidden Dissatisfaction     82
Politeness Bias             9
Name: count, dtype: int64

# # # Explainable Reasons****  

In [22]:
NEGATIVE_TRIGGERS = [
    "late", "rude", "slow", "bad", "cold", "delay",
    "খারাপ", "দেরি", "ঠান্ডা", "রূঢ়", "নোংরা"
]

def extract_reason(text):
    if not isinstance(text, str):
        return ""

    text_l = text.lower()
    reasons = [w for w in NEGATIVE_TRIGGERS if w in text_l]

    if reasons:
        return ", ".join(reasons[:3])
    else:
        return "Implicit dissatisfaction (tone-based)"

df_cleaned["explanation"] = df_cleaned.apply(
    lambda x: extract_reason(x["review_text"])
    if x["overall_sentiment"] == "negative"
    else "",
    axis=1
)
df_cleaned[
    df_cleaned["overall_sentiment"] == "negative"
][["review_text", "explanation"]].head(10)



,review_text,explanation
22,Cozy 5 star environment but the diversity of f...,Implicit dissatisfaction (tone-based)
28,"Not how you get reviews for a restaurant, I ho...",Implicit dissatisfaction (tone-based)
37,The atmosphere of this place and the service i...,Implicit dissatisfaction (tone-based)
51,The food is excellent. The candle light dinner...,Implicit dissatisfaction (tone-based)
56,"Let me start with the good things, their food....",late
62,Service and atmosphere were quite nice but I w...,Implicit dissatisfaction (tone-based)
82,A quite expensive restaurant which is located ...,bad
85,"The food was terrible. It was way too salty, t...",Implicit dissatisfaction (tone-based)
87,I had a lot of expectations from this place. T...,Implicit dissatisfaction (tone-based)
93,The food has lost its spark. Writing this with...,late


# # # Theme-Level Insight****  

In [23]:
theme_summary = (
    df_cleaned[df_cleaned["overall_sentiment"] == "negative"]
        .groupby("explanation")
        .size()
        .sort_values(ascending=False)
        .head(10)
)

theme_summary



explanation
Implicit dissatisfaction (tone-based)    146
bad                                       29
late                                      11
cold                                       8
slow                                       6
bad, cold                                  3
delay                                      2
slow, bad                                  2
late, cold                                 2
late, rude, bad                            1
dtype: int64

# # # Business-Ready Summary****  

In [24]:
print("🔍 KEY BUSINESS INSIGHTS\n")

print("Top Negative Drivers:")
for reason, count in theme_summary.items():
    print(f"- {reason}: {count} reviews")

conflict_rate = (
    df_cleaned["rating_sentiment_conflict"] != "No Conflict"
).mean() * 100

print(f"\nRating–Sentiment Conflict Rate: {conflict_rate:.2f}%")


🔍 KEY BUSINESS INSIGHTS

Top Negative Drivers:
- Implicit dissatisfaction (tone-based): 146 reviews
- bad: 29 reviews
- late: 11 reviews
- cold: 8 reviews
- slow: 6 reviews
- bad, cold: 3 reviews
- delay: 2 reviews
- slow, bad: 2 reviews
- late, cold: 2 reviews
- late, rude, bad: 1 reviews

Rating–Sentiment Conflict Rate: 18.02%


# Initialize Chroma Vector Store

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L12-v2",
    model_kwargs={"device": "cpu"}  # Force CPU to avoid CUDA OOM
)

# Prepare directory (Kaggle safe)
persist_dir = "/kaggle/working/chroma_store"
shutil.rmtree(persist_dir, ignore_errors=True)

# ✅ Create Chroma vector DB
vector_db = Chroma.from_documents(
    documents=docs,
    embedding=embedding,
    persist_directory=persist_dir,
    client_settings=Settings(anonymized_telemetry=False)
)

# Rag Based LLM for Quick Chat

In [ ]:
model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256)
llm = HuggingFacePipeline(pipeline=pipe)

print("✅ Flan-T5 loaded.")

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_db.as_retriever(search_kwargs={"k": 5}),
    return_source_documents=True
)

In [ ]:
questions = [
    "Which is the best coffee shop in Dhaka?",
    "Why this is the best coffee shop?",
    "Which coffee people buys the most?",
    "Why people buy that specific coffee?"
]

for question in questions:
    result = qa({"query": question})
    print(f"\n🧠 Q: {question}")
    print("📝 A:", result["result"])


In [ ]:
questions = [
    "Which is the best Kacchi restaurant in Gulshan?",
    "Why this restaurant is the best?",
    "Apart from Kacchi what else people order?",
    "What is the address?"
]

for question in questions:
    result = qa({"query": question})
    print(f"\n🧠 Q: {question}")
    print("📝 A:", result["result"])


# RAG LLM Evaluation 

In [ ]:
!pip uninstall -y ragas
!pip install --upgrade --no-cache-dir --force-reinstall \
    "ragas==0.0.18" \
    "pydantic<2.0" \
    "langchain<0.1.100" \
    "langchain-core<0.1.60" \
    "sentence-transformers<3.0.0" \
    "transformers<4.39.0" \
    "datasets>=2.10.0,<2.19.0"

In [ ]:
from langchain.schema import Document
from ragas.types import EvaluationDataset, QuestionAnswerPair

questions = [
    "What do people think about the chocolate cake in Gulshan?",
    "How is the service at Banani cafes?"
]

answers = [
    "People say the chocolate cake is rich and indulgent.",
    "Service in Banani is friendly and helpful."
]

ground_truths = [
    "Reviews say Gulshan chocolate cake is amazing and worth revisiting.",
    "Staff in Banani cafes are attentive and courteous."
]

contexts = [
    [Document(page_content="The Ultimate Chocolate Cake was rich and indulgent.")],
    [Document(page_content="The staff is polite, attentive, and always ready to help.")]
]

dataset = EvaluationDataset(
    question_answer_pairs=[
        QuestionAnswerPair(
            question=q,
            answer=a,
            contexts=c,
            ground_truth=g
        )
        for q, a, c, g in zip(questions, answers, contexts, ground_truths)
    ]
)


In [ ]:
from ragas.evaluation import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    answer_similarity,
)

results = evaluate(
    dataset=dataset,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall,
        answer_similarity,
    ]
)
results


In [ ]:
results_df = results.to_pandas()
results_df["faithfulness_%"] = (results_df["faithfulness"] * 100).round(1)
results_df["answer_relevancy_%"] = (results_df["answer_relevancy"] * 100).round(1)
results_df["context_precision_%"] = (results_df["context_precision"] * 100).round(1)
results_df["context_recall_%"] = (results_df["context_recall"] * 100).round(1)
results_df["answer_similarity_%"] = (results_df["answer_similarity"] * 100).round(1)

results_df[[
    "faithfulness_%", "answer_relevancy_%", "context_precision_%", "context_recall_%", "answer_similarity_%"
]]


# Aspect Based Sentiment Analysis

In [ ]:
!pip install -q transformers nltk
import pandas as pd
import numpy as np
import nltk
from transformers import pipeline
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

In [ ]:
#Clean and Prepare Data
df = df.dropna(subset=["Review Text", "Address", "Additional Info"])
df.loc[:, "Address"] = df["Address"].str.strip()
df.loc[:, "Additional Info"] = df["Additional Info"].str.strip()

df = df.copy()  # ✅ Ensure it's a true copy before assignment
df["Address"] = df["Address"].str.strip()
df["Additional Info"] = df["Additional Info"].str.strip()


In [ ]:
# Load Sentiment Analysis Model 
sentiment_pipeline = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest")


In [ ]:
aspects = {
    "food": ["food", "dish", "taste", "cake", "coffee", "flavor", "meal"],
    "service": ["staff", "service", "waiter", "behavior", "rude", "friendly"],
    "price": ["price", "cost", "expensive", "cheap", "worth", "value"],
    "atmosphere": ["ambiance", "environment", "decor", "music", "view", "lighting"]
}

In [ ]:
# Function for Aspect-Based Sentiment per Review
def analyze_aspects(text):
    sentences = sent_tokenize(text.lower())
    result = {}

    for aspect, keywords in aspects.items():
        aspect_sentences = [s for s in sentences if any(k in s for k in keywords)]
        if aspect_sentences:
            joined = " ".join(aspect_sentences)[:512]
            sentiment = sentiment_pipeline(joined)[0]
            label = sentiment['label']
            score = sentiment['score'] if label == "POSITIVE" else -sentiment['score'] if label == "NEGATIVE" else 0
            result[aspect] = score
        else:
            result[aspect] = np.nan
    return result

In [ ]:
aspect_sentiments = df["Additional Info"].apply(analyze_aspects)
aspect_df = pd.concat([df, pd.DataFrame(aspect_sentiments.tolist())], axis=1)

In [ ]:
city_summary = aspect_df.groupby("Address")[list(aspects.keys())].mean().sort_values("food", ascending=False)
print(city_summary.round(3))


In [ ]:
city_summary["food"].plot(kind="barh", title="🍽️ Food Sentiment by City", figsize=(10, 6), color="teal")

In [ ]:
coffee_df = aspect_df[aspect_df["Review Text"].str.lower().str.contains("coffee")]
coffee_city_sentiment = coffee_df.groupby("City")["food"].mean().sort_values(ascending=False)
print(coffee_city_sentiment.round(3))


In [ ]:
import re

def extract_aspects(text):
    if pd.isna(text):
        return {}

    result = {}
    patterns = {
        "food": r"Food:\s*(\d+)",
        "service": r"Service:\s*(\d+)",
        "ambiance": r"Atmosphere:\s*(\d+)",
        "price": r"Price per person.*?(\d{3,5})"
    }

    for aspect, pattern in patterns.items():
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            try:
                result[aspect] = float(match.group(1))
            except:
                result[aspect] = None
        else:
            result[aspect] = None

    return result


In [ ]:
aspect_ratings = df["Additional Info"].apply(extract_aspects)
aspect_scores_df = pd.DataFrame(aspect_ratings.tolist())
df_aspect = pd.concat([df, aspect_scores_df], axis=1)

In [ ]:
city_aspects = df_aspect.groupby("Address")[["food", "service", "ambiance", "price"]].mean().round(2)
print(city_aspects.sort_values("food", ascending=False))

In [ ]:
# Top cities by food rating
print(city_aspects["food"].sort_values(ascending=False).head())

# Cities with lowest service ratings
print(city_aspects["service"].sort_values().head())

# Cities with highest price per person
print(city_aspects["price"].sort_values(ascending=False).head())


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.heatmap(city_aspects, annot=True, cmap="YlGnBu")
plt.title("Aspect-Based Ratings by City")
plt.show()
